# Notebook pour les annotations automatiques des corpus

On importe tout d'abord les bibliothèques nécessaires

In [26]:
import regex as re
import pandas as pd

# Annotations des corpus de doublets complets

Après avoir nettoyé manuellement les corpus csv des entrées non-adaptés, on crée un corpus général en ajoutant une colonne "Corpus" indiquant la plateforme source de chaque entrée.

In [28]:
doc = pd.read_csv('reddit_nettoyé.csv')
doc["corpus"]="Reddit"
doc2 = pd.read_csv('twitter_nettoyé.csv')
doc2["corpus"]="Twitter"
doc3 = pd.read_csv('youtube_nettoyé.csv')
doc3["corpus"]="Youtube"

# Concaténation des trois fichiers
doc_final = pd.concat([doc, doc2, doc3], ignore_index=True)
doc_final.to_csv('corpusRTY.csv', index=False, sep=',')
#On crée un fichier excel pour une meilleure lisibilité
doc_final.to_excel('corpusRTY.xlsx', index=False)

## Annotation des conjonctions de coordination des doublets

On ajoute les colonnes Cat, Nb, Humain et Conj au corpus global et remplit la colonne "Conj".


In [29]:
docxlsx = pd.read_excel('corpusRTY.xlsx')

#Je crée quatre colonnes vides pour les éléments à annoter
docxlsx["Cat"]="/"
docxlsx["Humain"]="/"
docxlsx["Conj"]="/"
docxlsx["Nb"]="/"

#J'annote les conjonctions de coordination "et" et "ou" dans la colonne "Conj"
for i, valeur in docxlsx["keyword"].items():
    valeur_lower = valeur.lower()
    if " ou " in valeur_lower:
        docxlsx.loc[i, "Conj"] = "ou"
    elif " et " in valeur_lower:
        docxlsx.loc[i, "Conj"] = "et"
    else:
        docxlsx.loc[i, "Conj"] = ""

#Je vérifie la répartition des annotations
print(docxlsx["Conj"].value_counts())


Conj
et    2698
       995
ou     944
Name: count, dtype: int64


On ajoute le slash aux options possibles de conjonction et on vérifie

In [30]:
#idem que la cellule précédente avec le slash
for i, valeur in docxlsx["keyword"].items():
    valeur_lower = valeur.lower()
    if "/"  in valeur_lower:
        docxlsx.loc[i, "Conj"] = "/"
    else:
        pass

#Je vérifie la répartition des annotations
print(docxlsx["Conj"].value_counts())

#On affiche le pourcentage des annotations
pourcentageConj = len(docxlsx[docxlsx["Conj"]!=""])/len(docxlsx)*100
print("On a annoté ",pourcentageConj,"% du corpus.")

#J'enregistre le fichier des conjonctions annotées automatiquement
docxlsx.to_excel('corpus_annotéAUTO_conj.xlsx')

Conj
et    2698
/      953
ou     944
        42
Name: count, dtype: int64
On a annoté  99.09424196678887 % du corpus.


On annote manuellement les 42 entrées manquantes qui ne correspondent ni à "et", ni à "ou" ni au "/" et passe à l'annotation en nombre.

## Annotation du nombre des doublets

In [31]:
#Lecture du fichier avec les conjonctions intégralement annotées
docConj = pd.read_excel('corpus_annotéMANU_conj.xlsx')

#regex pour le pluriel et le singulier
pluriel = r'\b[a-zA-Z àâäéèêëîïôöùûüÿçÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇ]+[sxSX]\b\s*(et|ou|/|[.,-·]|\()\s*\b[a-zA-ZàâäéèêëîïôöùûüÿçÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇ]+[sSxX]\b(\))?'
singulier = r'\b[a-zA-ZàâäéèêëîïôöùûüÿçÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇ]+[^sS\s]\b\s*(et|ou|/|[.,-·]|\()\s*\b[a-zA-ZàâäéèêëîïôöùûüÿçÀÂÄÉÈÊËÎÏÔÖÙÛÜŸÇ]+[^sS\s]\b(\))?'

#On annote en fonction du nombre des keywords
for i, valeur in docConj["keyword"].items():
    valeur_lower = valeur.lower()
    if re.search(pluriel, str(valeur_lower)):
        docConj.loc[i, "Nb"] = "P"
    elif re.search(singulier, str(valeur_lower)):
        docConj.loc[i, "Nb"] = "S"
    else:
        docConj.loc[i, "Nb"] = "/"

#Je vérifie la répartition des annotations
print(docConj["Nb"].value_counts())

#On affiche le pourcentage des annotations
pourcentageNb = len(docConj[docConj["Nb"]!="/"])/len(docConj)*100
print("On a annoté ",pourcentageNb,"% du corpus.")

#J'enregistre le fichier avec le nombre annoté automatiquement
docConj.to_excel('corpus_annotéAUTO_conj_nb.xlsx')


Nb
P    2846
S    1745
/      46
Name: count, dtype: int64
On a annoté  99.00797929695923 % du corpus.


Nous annotons les 46 entrées restantes en modifiant S en SS pour les doublets avec deux termes au singulier et PP pour les doublets avec deux termes au pluriel. Nous indiquons SP pour les doublets avec le premier termeau singulier et le second au pluriel et inversement.


## Annotations des catégories grammaticales des doublets

In [33]:
docCat = pd.read_excel('corpus_annotéMANU_conj_nb.xlsx')

cellesceux = r"\b(celles|ceux)\s*(et|ou|[\/,.·\-]|\(\s*)\s*(celles|ceux)\)?\b"
celuicelle = r"\b(celui|celle)\s*(et|ou|[\/,.·\-]|\(\s*)\s*(celui|celle)\)?\b"
chacunchacune = r"\b(chacun(s)?|chacune(s)?)\s*(et|ou|[\/,.·\-]|\(\s*)\s*(chacun(s)?|chacune(s)?)\)?\b"
elleux = r"\b(eux|elles)\s*(et|ou|[\/,.·\-]|\(\s*)\s*(eux|elles)\)?\b"
iels = r"\b(il(s)?|elle(s)?)\b\s*(et|ou|\/|,|\.|\-|\·|\(\s*)\s*\b(il(s)?|elle(s)?)\b(\))?"
nombreuxses = r"\b(nombreuses|nombreux)\s*(et|ou|[\/,.·\-]|\(\s*)\s*(nombreuses|nombreux)\)?\b"
fr = r"\b((les\s+)?(français|française(s)?))\s*(et|ou|[\/,.·\-]|\(\s*)\s*((les\s+)?(français|française(s)?))\)?\b"
ellui = r"\b(elle|lui)\s*(et|ou|[\/,.·\-]|\(\s*)\s*(elle|lui)\)?\b"

for i, valeur in docCat["keyword"].items():
    valeur_lower = valeur.lower()
    if re.search(cellesceux, str(valeur_lower)):
        docCat.loc[i, "Cat"] = "P"
    elif re.search(celuicelle, str(valeur_lower)):
        docCat.loc[i, "Cat"] = "P"
    elif re.search(chacunchacune, str(valeur_lower)):
        docCat.loc[i, "Cat"] = "P"
    elif re.search(elleux, str(valeur_lower)):
        docCat.loc[i, "Cat"] = "P"
    elif re.search(iels, str(valeur_lower)):
        docCat.loc[i, "Cat"] = "P"
    elif re.search(ellui, str(valeur_lower)):
        docCat.loc[i, "Cat"] = "P"
    elif re.search(nombreuxses, str(valeur_lower)):
        docCat.loc[i, "Cat"] = "A"
    elif re.search(fr, str(valeur_lower)):
        docCat.loc[i, "Cat"] = "N"
    else:
        docCat.loc[i, "Cat"] = "/"

#On vérifie la répartition des annotations
print(docCat["Cat"].value_counts())

#On affiche le pourcentage des annotations
pourcentageCat = len(docCat[docCat["Cat"]!="/"])/len(docCat)*100
print("On a annoté ",pourcentageCat,"% du corpus.")

#On enregistre notre fichier annoté automatiquement avec la catégorie grammaticale
docCat.to_excel('corpus_annotéAUTO_conj_nb_cat.xlsx')

Cat
/    2319
P    2160
N     130
A      28
Name: count, dtype: int64
On a annoté  49.989217166271295 % du corpus.


## Annotation des catégories grammaticales restantes

On cherche si certains keywords peuvent être encore annotés automatiquement

In [15]:
# Créez un dictionnaire pour stocker les occurrences
occurrences = {}

# Parcourez les éléments de votre DataFrame
for i, valeur in docCat["keyword"].items():
    # Si la catégorie associée est "/"
    if docCat.loc[i, "Cat"] == "/":
        # Vérifiez si la valeur est déjà dans le dictionnaire
        if valeur in occurrences:
            # Incrémentez le compteur si la valeur est déjà présente
            occurrences[valeur] += 1
        else:
            # Ajoutez la valeur au dictionnaire avec un compteur de 1 si elle n'est pas encore présente
            occurrences[valeur] = 1

# Trier les occurrences par ordre décroissant
occurrences_triees = sorted(occurrences.items(), key=lambda item: item[1], reverse=True)

# Affichez les occurrences triées
for valeur, count in occurrences_triees:
    print(valeur,count)



toutes et tous 493
le / la 195
tous et toutes 173
un ou une 165
un / une 132
le ou la 104
ton / ta 62
travailleurs et travailleuses 35
acteurs et actrices 27
certains et certaines 27
Toutes et tous 22
ton ou ta 18
la / le 17
travailleuses et travailleurs 16
étudiants et étudiantes 15
candidates et candidats 15
auteurs et autrices 15
certains ou certaines 15
son ou sa 14
certaines et certains 14
candidats et candidates 13
actrices et acteurs 13
électeurs et électrices 13
Le ou la 13
électrices et électeurs 12
une ou un 12
une / un 11
ta / ton 10
Le / la 10
lectrices et lecteurs 9
Tous et toutes 9
lecteurs et lectrices 9
amies et amis 9
militantes et militants 8
collaborateurs et collaboratrices 8
chercheurs et chercheuses 8
joueurs et joueuses 8
Un ou une 7
cousins et cousines 7
demandeuses et demandeurs 6
chercheuses et chercheurs 6
habitantes et habitants 6
amis et amies 6
sénateurs et sénatrices 6
militants et militantes 6
sénatrices et sénateurs 6
influenceurs et influenceuses 5
age

On observe que les doublets "ton/ta" et ses variantes, "son/sa" et ses variantes peuvent être également annotés automatiquement comme des pronoms. On peut également considérer "travailleuses/travailleurs" et ses variantes (également en nombre) comme un doublet de noms et l'annoter en conséquences.

In [34]:
staon = r"\b((s|t)on|(t|s)a)\s*(et|ou|[\/,.·\-]|\(\s*)\s*((s|t)on|(s|t)a)\)?\b"
travail = r"\b(travailleur(s)?|travailleuse(s)?)\s*(et|ou|[\/,.·\-]|\(\s*)\s*(travailleur(s)?|travailleuse(s)?)\)?\b"

for i, valeur in docCat["keyword"].items():
    valeur_lower = valeur.lower()
    if re.search(staon, str(valeur_lower)):
        docCat.loc[i, "Cat"] = "P"
    elif re.search(travail, str(valeur_lower)):
        docCat.loc[i, "Cat"] = "N"
    else:
        pass

#On vérifie la répartition des annotations
print(docCat["Cat"].value_counts())

#On affiche le pourcentage de données annotées
pourcentage = len(docCat[docCat["Cat"]!="/"])/len(docCat)*100
print("On a annoté ",pourcentage,"% du corpus.")

#On enregistre notre fichier annoté automatiquement avec la catégorie grammaticale
docCat.to_excel('corpus_annotéAUTO_conj_nb_catV2.xlsx')

Cat
P    2274
/    2151
N     184
A      28
Name: count, dtype: int64
On a annoté  53.61224929911581 % du corpus.
